In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import time
import copy
import datetime

In [6]:
import torch 
from torch.nn import Conv2d, MaxPool2d, BatchNorm2d, Linear, Dropout, Module, CrossEntropyLoss, BatchNorm1d
from torch.nn.functional import sigmoid, relu, leaky_relu, tanh, softmax, max_pool2d
from torch.optim import Adam,RMSprop,SGD, lr_scheduler
from torch.autograd import backward
from torchvision.datasets import CIFAR100, CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
from torch.utils.tensorboard import SummaryWriter
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
import torchvision
print("Runinng on :", device,torch.__version__)

Runinng on : cuda:0 1.1.0


In [7]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = CIFAR10("../Data/",train=True,transform = transform,download=True)
test_dataset = CIFAR10("../Data/",train=False, transform = transform,download=True)

Files already downloaded and verified
Files already downloaded and verified


In [26]:
writer = SummaryWriter("./runs/cifar10_experiment_1")
batch_size = 1
train_dataloader = DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

In [27]:
print("Train Size",len(train_dataloader))
print("Test Size",len(test_dataloader))
dataloaders = {"train":train_dataloader, "val":test_dataloader}
dataset_sizes = {"train":len(train_dataset), "val":len(test_dataset)}

Train Size 50000
Test Size 10000


In [28]:
class CIFAR100(torch.nn.Module):
    
    def __init__(self):
        super(CIFAR100,self).__init__()
        #(feature_map - kernel + 2*padding)/stride + 1
        self.conv_1 = Conv2d(in_channels = 3, out_channels=32,kernel_size=3,padding=1)
        self.bn1 = BatchNorm2d(32)
        self.conv_2 = Conv2d(in_channels = 32, out_channels=64,kernel_size=3,stride=1,padding=1)
        self.bn2 = BatchNorm2d(64)
        self.dropout_1 = Dropout(p=0.8)
        
        self.conv_3 = Conv2d(in_channels = 64, out_channels=64,kernel_size=3,padding=1)
        self.bn3 = BatchNorm2d(64)
        self.conv_4 = Conv2d(in_channels = 64, out_channels=128,kernel_size=3, stride=1,padding=1)
        self.bn4 = BatchNorm2d(128)
        self.dropout_2 = Dropout(p=0.8)
        
        self.fc1 = Linear(in_features=128*8*8, out_features = 512)
        self.bn5 = BatchNorm1d(512)
        self.fc2 = Linear(in_features=512, out_features=10)
        
        
        
    def forward(self,x):
        x = relu(self.conv_1(x))
        x = self.bn1(x)
        x = relu(self.conv_2(x))
        x = self.bn2(x)
        x = max_pool2d(x,2)
        x = self.dropout_1(x)

        x = relu(self.conv_3(x))
        x = self.bn3(x)
        x = relu(self.conv_4(x))
        x = self.bn4(x)
        x = max_pool2d(x,2)
        x = self.dropout_2(x)
        x = x.view(-1, 128*8*8)
        x = self.fc1(x)
        x = self.bn5(x)
        x = self.fc2(x)
        x = softmax(x,dim=1)
        return x

In [34]:
model = CIFAR100()
model = model.to(device)


# model = torchvision.models.resnet18(pretrained=False)
# for param in model.parameters():
#     param.requires_grad = True
# num_ftrs = model.fc.in_features
# model.fc = Linear(num_ftrs, 10)
# model = model.to(device)

images,labels = iter(train_dataloader).next()
print(images.shape)
writer.add_graph(model,  torch.rand((3,32,32)).unsqueeze(0).cuda())
writer.close()
# writer.add_graph(model,images)
criterion = CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

torch.Size([1, 3, 32, 32])


In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            print(datetime.datetime.now())
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            count = 0
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
                count = count +1 
                
                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
#                     print(inputs.shape,outputs.shape)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
#                 if phase == "val":
#                     print(running_corrects)
#                     print(preds, labels.data)
                if count%100 == 0:
                    print("STEP :",count, phase.upper(),"LOSS: ",loss.item())
                    print('ACC :{:.4f}'.format(running_corrects.double()/(count*batch_size)))
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [ ]:
model_ft = train_model(model, criterion, optimizer, exp_lr_scheduler,
                       num_epochs=30)